In [29]:
import cv2
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import torch.nn.functional as F

In [30]:
class LightweightCNN(nn.Module):
    def __init__(self):
        super(LightweightCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1)
        self.maxpool1 = nn.MaxPool2d(kernel_size=4, stride=2)

        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=2, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=4, stride=2)
        
        self.conv3 = nn.Conv2d(32, 64, kernel_size=5, stride=2, padding=1)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(64, 32)
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(32, 6)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.maxpool1(x)
        x = F.relu(self.conv2(x))
        x = self.maxpool2(x)
        x = F.relu(self.conv3(x))
        x = self.maxpool3(x)

        # Reshape for fully connected layer
        x = torch.flatten(x, 1)
        
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

In [31]:
# Load an image from path and convert it to a tensor
def load_image(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    img = img.astype(np.float32) / 255.0

    # Down sample image to 160x120
    img = cv2.resize(img, (160, 120))

    print(img.shape)

    # Reshape the image to have channels as the first dimension
    img = img.transpose(2, 0, 1)

    print(img.shape)

    # Convert to tensor
    img = torch.from_numpy(img)

    # Add a batch dimension
    img = img.unsqueeze(0)

    print(img.shape)

    return img

# Load the model from path
def load_model(path):
    model = torch.load(path)

    return model


In [33]:
# Load the model
model = LightweightCNN()
model.load_state_dict(torch.load("/Users/jvelasquez/Virginia_Tech/Spring_2023/ECE_4806/ieee-robotics-2023-code/src/jetson_code/pedestal_classification/lightweight_net_color_orientation_v2.pth"))
model.eval()

LightweightCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1))
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=6, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

In [37]:
# Load the image
img = load_image("/Users/jvelasquez/Downloads/jpg/IMG_0757.jpg")

# Run the model
out = model(img)

print(out)

# Retrieve the index of the highest activation
_, pred = torch.max(out, 1)

if pred == 0:
    print("Green & Fallen")
elif pred == 1:
    print("Green & Standing")
elif pred == 2:
    print("Red & Fallen")
elif pred == 3:
    print("Red & Standing")
elif pred == 4:
    print("White & Fallen")
elif pred == 5:
    print("White & Standing")
else:
    print("Error")



(120, 160, 3)
(3, 120, 160)
torch.Size([1, 3, 120, 160])
tensor([[ 21.3938,  -1.4170, -11.1174, -13.1421,  33.8826, -28.8596]],
       grad_fn=<AddmmBackward0>)
White & Fallen
